In [ ]:
from datetime import datetime, timezone
import gzip
import json
import pandas as pd
import mysql.connector


class Generic:
    
    def __init__(self,file):
        self.db = db
        self.file=file
        i=1
        with gzip.open(self.file, 'r') as infile:
            for line in infile:
                if i<2:
                    self.line=line
                    self.convert()
                    i=i+1
                else:
                    break
        
    def convert(self):
        data = self.line
        new_str = data.decode('utf-8')
        d = json.loads(new_str)
        self.d = d
#         print(d,'\n')

    def connection(self):
        db = mysql.connector.connect(
                  host="localhost",
                  user="root",
                  database="sample",
                  password="suman"
                )
        self.db = db
                
        
class SpecificTask(Generic):
    
        
    def makedict(self,file):
        self.db=db
        super().__init__(file)
        
        d = self.d
        
        datelist = ['create_date','expire_date','last_update','offline_date']
        
        for x in datelist:
            print(d[x])
            
            if d[x] == None:
                val = d[x]
            else:
                val = datetime.fromtimestamp( d[x]/1000, tz=timezone.utc )
                d.update({x: str(val)})
        
       
        j1 = {'ad_id':d['ad_id'], 
              'customer_id':d['customer_id'],
             'realm_id':d['realm_id'],
             'stock_number':d['stock_number'],
             'year':d['year'],
             'trim_name':d['trim_name'],
             'n_1':d['n_1'],
             'n_3':d['n_3'],
             'price':d['price'],
             'create_date':d['create_date'],
             'expire_date':d['expire_date'],
             'last_update':d['last_update'],
             'offline_date':d['offline_date']}
        
        print(j1)
        self.j1 = j1
        

    def insert_table(self):
        j1 = self.j1
        mydb=self.db
        print(mydb)
        mycursor = mydb.cursor()
        sql = """INSERT INTO stock_db (ad_id,customer_id,realm_id,stock_number,year,trim_name,
                n_1,n_3,price,create_date,expire_date,last_update,offline_date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        val = (j1['ad_id'],j1['customer_id'],j1['realm_id'],j1['stock_number'],
               j1['year'],j1['trim_name'],j1['n_1'],j1['n_3'],j1['price'],j1['create_date'],
               j1['expire_date'],j1['last_update'],j1['offline_date'])
        mycursor.execute(sql, val)
        mydb.commit()
        print(mycursor.rowcount, "record inserted.")




ob = SpecificTask('/Users/suman/Downloads/listings_2020-10-01_5.json.gz')
ob.makedict('/Users/suman/Downloads/listings_2020-10-01_5.json.gz')
ob.insert_table()
